<a href="https://colab.research.google.com/github/seulmi0827/fininsight/blob/main/JACE/BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q bertopic
!pip install -q bertopic[visualization]

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk -y
!pip install konlpy
!pip install mecab-python
!apt-get install curl -y
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 전처리

In [ ]:
from datetime import datetime

def parse_datetime(date_string):
   try:
       return datetime.fromisoformat(date_string)
   except ValueError:
       print(f"Invalid date format: {date_string}")
       return None

In [ ]:
from konlpy.tag import Mecab
import pandas as pd

mecab = Mecab()

def preprocess(text):
    # 명사 추출
    nouns = mecab.nouns(text)

    # 형태소 분석
    pos_tagged = mecab.pos(text)

    # 동사 추출 (VV: 동사)
    verbs = [word for word, pos in pos_tagged if pos.startswith('VV')]

    # 형용사 추출 (VA: 형용사)
    adjectives = [word for word, pos in pos_tagged if pos.startswith('VA')]

    # 부사 추출 (MAG: 일반부사)
    adverbs = [word for word, pos in pos_tagged if pos == 'MAG']

    # 모든 단어 결합 (1글자 이상인 것만)
    all_words = nouns + verbs + adjectives + adverbs
    filtered_words = [word for word in all_words if len(word) > 1]

    # 공백으로 조인하여 문자열로 반환
    return filtered_words


In [ ]:
# 데이터프레임 로드
df = pd.read_csv("/content/drive/MyDrive/fin/삼성전자_10000.csv")

df['inp_date'] = df['inp_date'].apply(parse_datetime)

df['content'] = df['content'].fillna('').astype(str)
original_content = df['content'].tolist()

preprocessed_content = [preprocess(preprocessed_content) for preprocessed_content in original_content]
df['preprocessed_content'] = [' '.join(content) for content in preprocessed_content]

In [ ]:
len(df)

In [ ]:
df.columns.tolist()

In [ ]:
df[['inp_date', 'preprocessed_content']].head()

# Mecab과 SBERT를 이용한 Bertopic

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN

import torch
import numpy as np

import random

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

# SEED = 42
# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# if torch.cuda.is_available():
#   torch.cuda.manual_seed_all(SEED)
#   torch.backends.cudnn.deterministic = True
#   torch.backends.cudnn.benchmark = False

In [ ]:
# umap_model = UMAP(
#    n_neighbors=15,
#    n_components=5,
#    min_dist=0.0,
#    metric='cosine',
#    random_state=SEED
# )

# hdbscan_model = HDBSCAN(
#    min_cluster_size=10, # 1개 주제를 만드려면 최소 3개의 비슷한 문서가 있어야 한다는 말. 적은 문서에서 nr_topics의 지정값이 출력되지 않아서 이를 조정
#    metric='euclidean',
#    prediction_data=True, # 모델 초기화
#    gen_min_span_tree=True,
#    cluster_selection_method='eom'
# )


embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

vectorizer = CountVectorizer(stop_words=None) # TF-IDF

topic_model = BERTopic(
   language="korean",
   nr_topics=30,
   top_n_words=5,
   calculate_probabilities=True,
  #  umap_model=umap_model,
  #  hdbscan_model=hdbscan_model,
   embedding_model=embedding_model,
   vectorizer_model=vectorizer, # TF-IDF to c-TF-IDF
   verbose=True, # 로그 추적
)


content_for_topic = [' '.join(doc) for doc in preprocessed_content]
topics, probs = topic_model.fit_transform(content_for_topic)


# topics는 각 문서가 할당된 주요 토픽의 ID를 담은 배열입니다.
# probs는 각 문서가 모든 토픽에 대해 가지는 확률값을 담은 2D 배열입니다. 크기는 (문서 수 × 토픽 수)입니다.

# calculate_probabilities 옵션의 차이는 다음과 같습니다:
  # True로 설정 시:
  # 각 문서가 각 토픽에 속할 확률을 계산합니다
  # fit_transform()이 토픽 ID와 확률값 두 가지를 반환합니다: topics, probs = fit_transform()
  # 문서가 여러 토픽에 걸쳐 있는 정도를 알 수 있습니다
  # 계산 시간이 더 오래 걸립니다

  # False로 설정 시(기본값):
  # 각 문서를 가장 가능성 높은 하나의 토픽에만 할당합니다
  # fit_transform()이 토픽 ID만 반환합니다: topics = fit_transform()
  # 계산이 더 빠릅니다
  # 다중 토픽 정보는 얻을 수 없습니다

In [ ]:
print(type(topic_model.get_topic_info()))

In [ ]:
print(list(topic_model.get_topic_info()))

In [ ]:
# index : 토픽 인덱스는 단순히 모델이 내부적으로 할당한 식별자일 뿐, 중요성이나 순위를 나타내지 않습니다.
# Topic: 토픽 ID 번호 (-1은 아웃라이어 토픽)
# Count: 토탈 문서 수 중, 각 토픽에 할당된 문서 수
# Name: 토픽의 자동 생성된 이름 (주요 키워드로 구성)
# Representation: 토픽을 대표하는 주요 단어들 (top_n_words=k에 해당)
# Representative_Docs: 각 토픽을 가장 잘 대표하는 문서 예시, Representative_Docs에서 추출된 정보로 Representation을 만드는 게 아님!!!

# Topic 아웃라이너 : BERTopic에서 토픽 ID가 -1로 표시되는 특별한 분류
# 주요 토픽들에 잘 맞지 않는 문서들의 집합 (일관된 주제가 없거나 여러 주제가 혼합된 문서들이 포함, 아주 독특한 내용을 가진 문서들이 포함)
# BERTopic은 HDBSCAN 클러스터링을 사용하는데, 이 알고리즘은 노이즈가 많거나 데이터 밀도가 낮은 영역의 포인트를 아웃라이어로 분류

In [ ]:
print("총 문서 수 : ", topic_model.get_topic_info()['Count'].sum())

In [ ]:
topic_df = topic_model.get_topic_info()
topic_df = topic_df[topic_df['Topic'] != -1][['Topic', 'Representation']]
topic_df['Representation'] = topic_df['Representation'].apply(lambda x: ', '.join(x))
topic_df[['Topic', 'Representation']]
topic_df.head()

In [ ]:
df['topic_id'] = topics
df[['inp_date', 'preprocessed_content', 'topic_id']].head()

In [ ]:
df['topic_keywords'] = df['topic_id'].map(dict(zip(topic_df['Topic'], topic_df['Representation'])))
# Topic은 모델에서 나온 "토픽 키워드에 대한 ID", Representation은 "토픽 키워드에 대한 ID"에 해당하는 밸류(실제 토픽 키워드)

df[['inp_date', 'preprocessed_content', 'topic_keywords']]

In [ ]:
topic_keyword_distribution = df.groupby([df['inp_date'].dt.date, 'topic_keywords']).size().unstack(fill_value=0)


import plotly.graph_objs as go

traces = []
for column in topic_keyword_distribution.columns:
    trace = go.Scatter(
        x=topic_keyword_distribution.index,
        y=topic_keyword_distribution[column],
        mode='lines+markers',
        name=column
    )
    traces.append(trace)

layout = go.Layout(
    title='상승 키워드 분석',
    xaxis={'title': '날짜'},
    yaxis={'title': '문서 수'}
)

fig = go.Figure(data=traces, layout=layout)
fig.show()


In [ ]:
barchart = topic_model.visualize_barchart()
barchart.show()
print('\n')

heatmap = topic_model.visualize_heatmap()
heatmap.show()
print('\n')

hierarchy = topic_model.visualize_hierarchy()
hierarchy.show()
print('\n')

term_rank = topic_model.visualize_term_rank()
term_rank.show()
print('\n')

topics = topic_model.visualize_topics()
topics.show()